In [1]:
import psycopg2
import pandas as pd

In [17]:
def skeleton_1(where_str, params):
    con = psycopg2.connect(dbname='gpatterns', user='postgres',
                       password='postgres', host='localhost')
    comand = "select " + \
                "main.s_cl, main.animate, main.gender, main.number, main.case_morph, main.reflection, main.perfective, main.transitive, main.person, main.tense, main.voice, main.degree, main.static, " + \
                 "dep.s_cl, dep.animate, dep.gender, dep.number, dep.case_morph, dep.reflection, dep.perfective, dep.transitive, dep.person, dep.tense, dep.voice, dep.degree, dep.static, " + \
                 "gp.mark " + \
        "from gpattern_2_level as gp " + \
        "left join morph_constraints as main " + \
        "on main.id = main_morph " + \
        "left join morph_constraints as dep " + \
        "on dep.id = dep_morph " + \
        where_str
    cursor = con.cursor()
    cursor.execute(comand, params)
    ans = cursor.fetchall()
    cursor.close()
    con.close()
    names = ['s_cl', 'animate', 'gender', 'number', 'case_morph', 'reflection', 'perfective', 'transitive', 'person', 'tense', 'voice', 'degree', 'static']
    main_names = list(map(lambda x: 'main_' + x, names))
    dep_names = list(map(lambda x: 'dep_' + x, names))
    df = pd.DataFrame(ans, columns = main_names + dep_names + ['mark'])
    for column in df.columns:
        if (df[column] == 'not_imp').all():
            df.drop(column, axis='columns', inplace=True)
    return df

In [18]:
def skeleton_2(where_str, params):
    con = psycopg2.connect(dbname='gpatterns', user='postgres',
                       password='postgres', host='localhost')
    comand = "select main_word.name, " + \
                "main.s_cl, main.animate, main.gender, main.number, main.case_morph, main.reflection, main.perfective, main.transitive, main.person, main.tense, main.voice, main.degree, main.static, " + \
                 "dep.s_cl, dep.animate, dep.gender, dep.number, dep.case_morph, dep.reflection, dep.perfective, dep.transitive, dep.person, dep.tense, dep.voice, dep.degree, dep.static, " + \
                 "gp.mark " + \
        "from gpattern_2_level as gp " + \
        "left join morph_constraints as main " + \
        "on main.id = main_morph " + \
        "left join morph_constraints as dep " + \
        "on dep.id = dep_morph " + \
        "left join word as main_word " + \
        "on main_word.id = main_word " + \
        where_str
    cursor = con.cursor()
    cursor.execute(comand, params)
    ans = cursor.fetchall()
    cursor.close()
    con.close()
    names = ['s_cl', 'animate', 'gender', 'number', 'case_morph', 'reflection', 'perfective', 'transitive', 'person', 'tense', 'voice', 'degree', 'static']
    main_names = list(map(lambda x: 'main_' + x, names))
    dep_names = list(map(lambda x: 'dep_' + x, names))
    df = pd.DataFrame(ans, columns = ['main_word'] + main_names + dep_names + ['mark'])
    for column in df.columns:
        if (df[column] == 'not_imp').all():
            df.drop(column, axis='columns', inplace=True)
    return df

In [19]:
def skeleton_3(where_str, params):
    con = psycopg2.connect(dbname='gpatterns', user='postgres',
                       password='postgres', host='localhost')
    comand = "select main_word.name, dep_word.name, " + \
                "main.s_cl, main.animate, main.gender, main.number, main.case_morph, main.reflection, main.perfective, main.transitive, main.person, main.tense, main.voice, main.degree, main.static, " + \
                 "dep.s_cl, dep.animate, dep.gender, dep.number, dep.case_morph, dep.reflection, dep.perfective, dep.transitive, dep.person, dep.tense, dep.voice, dep.degree, dep.static, " + \
                 "gp.mark " + \
        "from gpattern_3_level as gp " + \
        "left join morph_constraints as main " + \
        "on main.id = main_morph " + \
        "left join morph_constraints as dep " + \
        "on dep.id = dep_morph " + \
        "left join word as main_word " + \
        "on main_word.id = main_word " + \
        "left join word as dep_word " + \
        "on dep_word.id = dep_word " + \
        where_str
    cursor = con.cursor()
    cursor.execute(comand, params)
    ans = cursor.fetchall()
    cursor.close()
    con.close()
    names = ['s_cl', 'animate', 'gender', 'number', 'case_morph', 'reflection', 'perfective', 'transitive', 'person', 'tense', 'voice', 'degree', 'static']
    main_names = list(map(lambda x: 'main_' + x, names))
    dep_names = list(map(lambda x: 'dep_' + x, names))
    df = pd.DataFrame(ans, columns = ['main_word', 'dep_word'] + main_names + dep_names + ['mark'])
    for column in df.columns:
        if (df[column] == 'not_imp').all():
            df.drop(column, axis='columns', inplace=True)
    return df

In [20]:
from Types import *

In [23]:
def createWhere(mainMorphParams, depMorphParams, mainWordParam, depWordParam):
    if mainWordParam:
        whereMainWord = "main_word.name = %s"
        if depWordParam:
            whereDepWord = "dep_word.name = %s "
            res = whereMainWord + " and " + whereDepWord
            params = (mainWordParam, depWordParam)
        else:
            res = whereMainWord
            params = (mainWordParam, )
    else:
        if depWordParam:
            whereDepWord = "dep_word.name = %s "
            res =  whereDepWord
            params = (depWordParam, )
        else:
            res = ""
            params = ()
            
    if depMorphParams:
        whereDepMorph = ""
        for i in range(len(depMorphParams)):
            curParam = depMorphParams[i]
            s1 = "dep." +  dictField[curParam] + " = " + "'" + curParam + "'"
            whereDepMorph += s1
            if i != len(depMorphParams) - 1:
                whereDepMorph += " and "
        if res != "":
            res = whereDepMorph + " and " + res
        else:
            res = whereDepMorph

    if mainMorphParams:
        whereMainMorph = ""
        for i in range(len(mainMorphParams)):
            curParam = mainMorphParams[i]
            s1 = "main." + dictField[curParam] + " = " + "'" + curParam + "'"
            whereMainMorph += s1
            if i != len(mainMorphParams) - 1:
                whereMainMorph += " and "
        if res != "":
            res = whereMainMorph + " and " + res
        else:
            res = whereMainMorph
    if res != "":
        res = "where " + res
    return (res, params)
    

In [29]:
def itog_select(level, mainMorphParams = None, depMorphParams = None, mainWordParam = None, depWordParam = None):
    where, params = createWhere(mainMorphParams, depMorphParams, mainWordParam, depWordParam)
    if level == 1:
        return skeleton_1(where, params)
    if level == 2:
        return skeleton_2(where, params)
    return skeleton_3(where, params)

In [30]:
itog_select(3, mainWordParam = 'взрослый')

/home/derin/anaconda3/envs/rulang-syntax-analyzer/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,main_word,dep_word,main_s_cl,main_animate,main_gender,main_number,main_case_morph,dep_s_cl,dep_animate,dep_gender,dep_number,dep_case_morph,dep_perfective,dep_tense,mark
0,взрослый,весь,noun,animate,male,plural,genitive,adjective,not_imp,not_imp,plural,genitive,not_imp,not_imp,1
1,взрослый,ребёнок,adjective,not_imp,not_imp,plural,nominative,noun,animate,male,plural,nominative,not_imp,not_imp,1
2,взрослый,учитель,noun,animate,male,plural,genitive,noun,animate,male,plural,genitive,not_imp,not_imp,1
3,взрослый,старик,noun,animate,male,single,instrumental,noun,animate,male,single,instrumental,not_imp,not_imp,1
4,взрослый,полноправный,noun,animate,male,plural,accusative,adjective,animate,not_imp,plural,accusative,not_imp,not_imp,1
5,взрослый,запутаться,noun,animate,male,plural,nominative,participle,not_imp,not_imp,plural,nominative,perfective,past,1
6,взрослый,очень,adjective,not_imp,male,single,nominative,adverb,not_imp,not_imp,not_imp,not_imp,not_imp,not_imp,1
7,взрослый,самый,adjective,not_imp,male,single,nominative,adjective,not_imp,male,single,nominative,not_imp,not_imp,1
8,взрослый,подзабыть,noun,animate,male,plural,dative,participle,not_imp,not_imp,plural,dative,perfective,past,1


TypeError: skeleton_1() missing 1 required positional argument: 'params'

In [12]:
ans1[ans1['main_case_morph'] == 'accusative'][ans1['dep_s_cl'] == 'adjective'][ans1['dep_number'] == 'plural']

TypeError: tuple indices must be integers or slices, not str